In [51]:
import tensorflow as tf
import os
import cv2
import pandas as pd
import numpy as np

In [52]:
# 이미지 데이터와 라벨을 저장할 리스트 초기화
data = []
labels = []

def make_dataset(dir_path, label):
    image_files = [f for f in os.listdir(dir_path) if f.endswith('.png')]
    
    # 이미지 파일 반복 처리
    for image_file in image_files:
        # 이미지 파일 경로 생성
        image_path = os.path.join(dir_path, image_file)
        
        # 이미지 읽기 (OpenCV 사용)
        image = cv2.imread(image_path)
        
        # 이미지 크기 조정 (필요한 경우)
        # image = cv2.resize(image, (width, height))
        
        # 이미지 데이터와 라벨 추가
        data.append(image)
        labels.append(label)  # 클래스 라벨을 적절하게 설정해야 합니다.



In [53]:
data_dir_0 = "C:/ssafy/candle/0/"
data_dir_1 = "C:/ssafy/candle/1/"

make_dataset(data_dir_0, 0)
make_dataset(data_dir_1, 1)

# 데이터와 라벨을 NumPy 배열로 변환
data = np.array(data)
labels = np.array(labels)

# 라벨을 원-핫 인코딩 (필요한 경우)
# labels = to_categorical(labels)
print(data.shape)
print(labels.shape)

(22787, 100, 100, 3)
(22787,)


In [54]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [55]:
# 데이터 분할
X_train, X_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 데이터 스케일링
X_train = X_train.astype('float32') / 255
X_val = X_val.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# 클래스 수 계산
num_classes = len(np.unique(labels))

# CNN 모델 정의
cnn_model = Sequential()
cnn_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
cnn_model.add(MaxPooling2D((2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
epochs = 3
batch_size = 64

history = cnn_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_val, y_val))

Epoch 1/3
214/214 [==============================] - 50s 231ms/step - loss: 0.7558 - accuracy: 0.5403 - val_loss: 0.6792 - val_accuracy: 0.5537
Epoch 2/3
214/214 [==============================] - 47s 222ms/step - loss: 0.6698 - accuracy: 0.5709 - val_loss: 0.6801 - val_accuracy: 0.5438
Epoch 3/3
214/214 [==============================] - 48s 224ms/step - loss: 0.6614 - accuracy: 0.5889 - val_loss: 0.6822 - val_accuracy: 0.5523


In [56]:
kospi_200 = pd.read_csv('kospi200.csv', encoding='EUC-KR')
kospi = pd.DataFrame(kospi_200)
kospi_dict = {}

# 코스피 200 종목코드, 종목명 딕셔너리 생성
for index, row in kospi.iterrows():
    code = str(row['종목코드'])
    length = len(code)
    if length < 6:
         for _ in range (6-length):
             code = '0' + code
    kospi_dict[code] = row['종목명']
print(kospi_dict)

{'005930': '삼성전자', '373220': 'LG에너지솔루션', '000660': 'SK하이닉스', '207940': '삼성바이오로직스', '005490': 'POSCO홀딩스', '051910': 'LG화학', '006400': '삼성SDI', '005380': '현대차', '035420': 'NAVER', '003670': '포스코퓨처엠', '000270': '기아', '105560': 'KB금융', '068270': '셀트리온', '012330': '현대모비스', '035720': '카카오', '028260': '삼성물산', '055550': '신한지주', '066570': 'LG전자', '096770': 'SK이노베이션', '047050': '포스코인터내셔널', '032830': '삼성생명', '003550': 'LG', '000810': '삼성화재', '033780': 'KT&G', '323410': '카카오뱅크', '086790': '하나금융지주', '015760': '한국전력', '329180': 'HD현대중공업', '138040': '메리츠금융지주', '018260': '삼성에스디에스', '034020': '두산에너빌리티', '034730': 'SK', '017670': 'SK텔레콤', '010130': '고려아연', '352820': '하이브', '009150': '삼성전기', '316140': '우리금융지주', '010950': 'S-Oil', '024110': '기업은행', '009540': 'HD한국조선해양', '003490': '대한항공', '011200': 'HMM', '030200': 'KT', '001570': '금양', '090430': '아모레퍼시픽', '042660': '한화오션', '259960': '크래프톤', '010140': '삼성중공업', '051900': 'LG생활건강', '326030': 'SK바이오팜', '086280': '현대글로비스', '028050': '삼성엔지니어링', '000100': '유한양행'

In [57]:
import FinanceDataReader as fdr
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick2_ohlc

# 최근 날짜의 주가 상승/하락 리스트
recent = []

for code in kospi_dict.keys():
    df = fdr.DataReader(code, '2023-08-01', '2023-09-19')
    

    # 실제 결과가 상승이면 1, 하락이면 0
    data = df.iloc[df.shape[0] - 11: df.shape[0] - 1]
    if df.iloc[df.shape[0] - 2]['Close'] > df.iloc[df.shape[0] - 1]['Close']:
        recent.append(0)
    else:
        recent.append(1)
        
    fig = plt.figure(figsize=(1,1))
    ax = fig.add_subplot(1, 1, 1)
    candlestick2_ohlc(ax, data['Open'], data['High'],
                        data['Low'], data['Close'], width=0.7, colorup='r', colordown='b')
    ax.grid(False)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.axis('off')
    plt.tight_layout()
    # 최근 10일의 캔들차트 저장
    plt.savefig('C:/ssafy/candle/recent/{}.png'.format(code), dpi=fig.dpi)
    plt.close()

In [66]:
# 최근 10일 캔들차트 가져와 테스트
recent_data_dir = "C:/ssafy/candle/recent/"
test_data = []
image_files = [f for f in os.listdir(recent_data_dir) if f.endswith('.png')]
    
# 이미지 파일 반복 처리
for image_file in image_files:
    # 이미지 파일 경로 생성
    image_path = os.path.join(recent_data_dir, image_file)
        
    # 이미지 읽기 (OpenCV 사용)
    image = cv2.imread(image_path)
        
    # 이미지 데이터추가
    test_data.append(image)

test_data = np.array(test_data)
print(test_data.shape)
test_data = test_data.astype('float32') / 255
prediction = cnn_model.predict(test_data)

acc = 0
for i in range (200):
    pre_value = 1
    if prediction[i][0] < 0.5:
        pre_value = 0
    if recent[i] == pre_value:
        acc += 1
    print(prediction[i][0])
print("정확도 : " + str(acc/200))

(200, 100, 100, 3)
7/7 [==============================] - 0s 11ms/step
0.4225176
0.3360302
0.38538444
0.46203148
0.31248233
0.39733222
0.5824964
0.39184964
0.37800282
0.34137446
0.39331672
0.5560453
0.37965623
0.3874311
0.43186596
0.38079557
0.431855
0.36401418
0.4582573
0.52966124
0.4140817
0.45841068
0.44548172
0.29027566
0.5504588
0.55089474
0.33702213
0.4587199
0.48028636
0.52964455
0.38161555
0.43343735
0.4841247
0.40778732
0.5489435
0.5250516
0.6057477
0.42615053
0.5361266
0.48820633
0.45756346
0.53905207
0.4916525
0.46561
0.5215897
0.41652852
0.5133128
0.5212848
0.44705623
0.6300834
0.5014215
0.3731375
0.508221
0.5235979
0.39605886
0.51303345
0.41229743
0.29811308
0.4937791
0.4899938
0.33847484
0.40349784
0.59962857
0.60110533
0.43039668
0.5201397
0.50356364
0.46284014
0.3983171
0.46144506
0.60695183
0.45126134
0.4668884
0.33890548
0.6606623
0.47898602
0.48152947
0.65897256
0.5746723
0.36405978
0.5254395
0.36439168
0.6158021
0.4425982
0.42053694
0.59239054
0.33626306
0.62710804
